# Fine-tuning Whisper with Hugging Face

In [1]:
"""
! pip install dataclasses
! pip install transformers[torch]
! pip install tensorboard
"""

'\n! pip install dataclasses\n! pip install transformers[torch]\n! pip install tensorboard\n'

# 0. Import Libraries

In [1]:
# 1
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration

from datasets import load_dataset
import time

# 1. Load Pretrained Model

In [2]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"

processor_small = WhisperProcessor.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")
model_small = WhisperForConditionalGeneration.from_pretrained("Taeyeun72/whisper-small-denoising_all") # Parameters: 244M

In [3]:
model_small

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [4]:
# 앞으로 사용할 model size는 small이다.
model = model_small
processor = processor_small

# 2. Load Dataset

- 바로 아래 코드는 <u>**실행 시간이 오래 걸리니**</u> 실행에 주의할 것.

In [15]:
from datasets import load_dataset
dataset = load_dataset("audiofolder", data_dir="./Dataset_Denoising_all_Combination")

Resolving data files:   0%|          | 0/307783 [00:00<?, ?it/s]

Computing checksums:  24%|##4       | 66026/272782 [00:05<00:15, 13203.07it/s]

Extracting data files:   0%|          | 0/272782 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
dataset['train'][0]

{'audio': {'path': 'C:/Users/Poco/Jupyter/03_2023_Fall/03_CapstonDesignInComputerScience/Dataset_Denoising_all_Combination/01.데이터/2.Validation/원천데이터/01.가전소음/01.세탁기,건조기/01_01_000252_210722_SN_1.wav',
  'array': array([ 0.03094482,  0.02258301,  0.02154541, ..., -0.00918579,
         -0.01013184, -0.00524902]),
  'sampling_rate': 16000},
 'text': '미안한데 네가 무슨 말 하는지 잘 모르겠어. 그리고 네가 학교 생활을 어떻게 해야 될지 왜 나한테 물어봐. 그런 건 네이버 지식인에다 물어보든지. 여기 그리고 기숙사 방인데. 조용히 좀 해. 세탁기 소리보다. 네 목소리가 더 커. 너 무슨 소문이 났다고 그러는데?'}

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 35000
    })
})

## 2.1. Inference Test

In [15]:
import time
import librosa

def inference(model, processor, audio): # audio ex: dataset['train'][0]
    start = time.time()

    forced_decoder_ids = processor.get_decoder_prompt_ids(language="korean", task="transcribe")
    
    y, s = librosa.load(audio['audio']['path'], sr=16000) # sampling rate 변경 (44100 -> 16000)
    print(f"Audio path: {audio['audio']['path']}")

    # 해당 모델은 최대 30초의 데이터만을 변환할 수 있음.

    input_features = processor(y, sampling_rate=16000, return_tensors="pt").input_features

    model.to('cpu')
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)

    end = time.time()
    
    print(f"Predicted:{processor.batch_decode(predicted_ids, skip_special_tokens=True)}")
    print(f"Answer:{audio['text']}")
    print(f"Inference time:{end-start}s")

In [16]:
inference(model_small, processor_small, dataset['train'][0])

Audio path: C:/Users/Poco/Jupyter/03_2023_Fall/03_CapstonDesignInComputerScience/Saveset/01.데이터/2.Validation/원천데이터/01.가전소음/01.세탁기,건조기/01_01_004107_210824_SD_33.wav
Predicted:[' 맞아 이제 돈도 많은 사람으로 오해할 수도 있잖아 그 여자분은 우리도 가끔 고시를 생각하면서 금전적인 걸 생각하면 금전적인 건 중요하지']
Answer:맞아. 이제 돈도 많은 사람으로 오해할 수도 있잖아.그 여자분은. 우리도 가끔 고 씨를 생각하면서. 금전적인 걸 생각하면. 금전적인 건 중요하지.
Inference time:3.9063498973846436s


In [33]:
inference(model_large_v2, processor_large_v2, dataset['train'][0])

Audio path: C:/Users/Poco/Jupyter/03_2023_Fall/00_HuggingFace/Dataset/train/[원천]10.낙상_1/10.낙상_1086706_label.wav
Predicted:[' 헛디뎌서 넘어졌어요.']
Answer:헛딛어서 넘어졌어요
Inference time:15.763933181762695s


# 3. Prepare Data

- Reference 1: `https://huggingface.co/blog/fine-tune-whisper`
- Reference 2: `https://velog.io/@mino0121/NLP-OpenAI-Whisper-Fine-tuning-for-Korean-ASR-with-HuggingFace-Transformers#prepare-data`

## 3.1. Load WhisperFeatureExtractor and WhisperTokenizer

In [5]:
from transformers import WhisperFeatureExtractor

# 패딩과 스펙트로그램 변환을 수행한다.
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [6]:
from transformers import WhisperTokenizer

# 토큰화를 수행한다.
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

In [14]:
# tokenizer의 수행을 테스트해보자.
input_str = dataset["train"][0]["text"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}") # True가 나오면 정상!

Input:                 야 같은 멤버인데 어떻게 그러냐. 내가 그렇게 하면은. 우리 그룹 완전 망해. 그래도 우리 먹고 살아야 되는데. 아니 안 그래도 그 오빠가 인기가 많으니까. 내가 뭐 어떻게 할 수 있나. 그 오빠 뭐 막 광고 찍고 이러면 그것도 다 우리한테 배분을 해 주잖아. 아니 그 오빠 여자친구. 사실
Decoded w/ special:    <|startoftranscript|><|ko|><|transcribe|><|notimestamps|>야 같은 멤버인데 어떻게 그러냐. 내가 그렇게 하면은. 우리 그룹 완전 망해. 그래도 우리 먹고 살아야 되는데. 아니 안 그래도 그 오빠가 인기가 많으니까. 내가 뭐 어떻게 할 수 있나. 그 오빠 뭐 막 광고 찍고 이러면 그것도 다 우리한테 배분을 해 주잖아. 아니 그 오빠 여자친구. 사실<|endoftext|>
Decoded w/out special: 야 같은 멤버인데 어떻게 그러냐. 내가 그렇게 하면은. 우리 그룹 완전 망해. 그래도 우리 먹고 살아야 되는데. 아니 안 그래도 그 오빠가 인기가 많으니까. 내가 뭐 어떻게 할 수 있나. 그 오빠 뭐 막 광고 찍고 이러면 그것도 다 우리한테 배분을 해 주잖아. 아니 그 오빠 여자친구. 사실
Are equal:             True


In [15]:
tokenizer.decode([14562])

'걸'

## 3.2. Preprocessing

In [20]:
# 먼저 원래의 데이터셋을 확인해보자.
dataset['train'][0] # sampling_rate이 44100으로 설정되어있다.

{'audio': {'path': 'C:/Users/Poco/Jupyter/03_2023_Fall/03_CapstonDesignInComputerScience/Dataset_Denoising_all_Combination/01.데이터/2.Validation/원천데이터/01.가전소음/01.세탁기,건조기/01_01_000252_210722_SN_1.wav',
  'array': array([ 0.03094482,  0.02258301,  0.02154541, ..., -0.00918579,
         -0.01013184, -0.00524902]),
  'sampling_rate': 16000},
 'text': '미안한데 네가 무슨 말 하는지 잘 모르겠어. 그리고 네가 학교 생활을 어떻게 해야 될지 왜 나한테 물어봐. 그런 건 네이버 지식인에다 물어보든지. 여기 그리고 기숙사 방인데. 조용히 좀 해. 세탁기 소리보다. 네 목소리가 더 커. 너 무슨 소문이 났다고 그러는데?'}

In [21]:
from datasets import Audio

# 오디오 샘플을 로드할 때 resampling(sampling rate 조정)을 수행한다.
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [22]:
# 변환된 데이터셋을 확인해보자.
dataset["train"][0] # sampling_rate이 16000으로 변환되었다.

{'audio': {'path': 'C:/Users/Poco/Jupyter/03_2023_Fall/03_CapstonDesignInComputerScience/Dataset_Denoising_all_Combination/01.데이터/2.Validation/원천데이터/01.가전소음/01.세탁기,건조기/01_01_000252_210722_SN_1.wav',
  'array': array([ 0.03094482,  0.02258301,  0.02154541, ..., -0.00918579,
         -0.01013184, -0.00524902]),
  'sampling_rate': 16000},
 'text': '미안한데 네가 무슨 말 하는지 잘 모르겠어. 그리고 네가 학교 생활을 어떻게 해야 될지 왜 나한테 물어봐. 그런 건 네이버 지식인에다 물어보든지. 여기 그리고 기숙사 방인데. 조용히 좀 해. 세탁기 소리보다. 네 목소리가 더 커. 너 무슨 소문이 났다고 그러는데?'}

In [23]:
# 데이터셋을 학습에 알맞게 준비해보자.
def prepare_dataset(batch):
    # 데이터셋(batch)에서 오디오 정보를 추출한다. (Resampling 포함: 48kHz -> 16kHz)
    audio = batch["audio"]

    # Log-melspectrogram feature를 batch 데이터 정보에 추가한다.
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # text를 토큰화한 id들을 batch 데이터 정보에 추가한다.
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 35000
    })
})

- 바로 아래 코드는 <u>**실행 시간이 <span style="color:red">상당히 오래</span> 걸리니**</u> 실행에 주의할 것! (약 40분 이상)

In [25]:
prepared_dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=None)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

- prepared_dataset의 구조를 확인해보자.

In [26]:
prepared_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 35000
    })
})

In [27]:
prepared_dataset['train'][0]

{'input_features': [[0.49805718660354614,
   0.413202166557312,
   0.3720576763153076,
   0.27397650480270386,
   0.04448467493057251,
   0.3434007167816162,
   0.32546567916870117,
   0.3744339942932129,
   0.4624946713447571,
   0.3860541582107544,
   0.39069658517837524,
   0.3779710531234741,
   0.4230825901031494,
   0.43657368421554565,
   0.2748916745185852,
   0.16267144680023193,
   0.328402042388916,
   0.44766074419021606,
   0.47623372077941895,
   0.514296293258667,
   0.4454503059387207,
   0.10038381814956665,
   0.3929978013038635,
   0.24777203798294067,
   0.3478708863258362,
   0.18978750705718994,
   0.477542519569397,
   0.3687082529067993,
   0.3983153700828552,
   0.35959112644195557,
   0.37757056951522827,
   0.4719177484512329,
   0.4296221137046814,
   0.33787238597869873,
   0.26411908864974976,
   0.31679224967956543,
   0.4100288152694702,
   0.42033135890960693,
   0.4819566011428833,
   0.45025521516799927,
   0.5032576322555542,
   0.42212051153182983,


In [28]:
len(prepared_dataset['train'][0]['input_features']), len(prepared_dataset['train'][0]['labels'])

(80, 88)

In [29]:
# 앞으로 전처리 과정을 생략하기 위해 prepared_dataset을 local에 저장해둔다.
prepared_dataset.save_to_disk("./prepared_dataset_Denoising_all_Combination")

Saving the dataset (0/68 shards):   0%|          | 0/35000 [00:00<?, ? examples/s]

- 만약 위의 과정을 이미 수행했다면, 앞으로는 다음 코드만을 실행시키면 된다.

In [7]:
prepared_dataset_1 = load_dataset("./prepared_dataset_Denoising_all")

Resolving data files:   0%|          | 0/69 [00:00<?, ?it/s]

In [8]:
prepared_dataset_2 = load_dataset("./prepared_New_Dataset_Denoising")

In [9]:
# train dataset과 test dataset으로 분리한다.
split_dataset = prepared_dataset_1['train'].train_test_split(test_size=0.01, seed=42)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 34650
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 350
    })
})

In [10]:
test_dataset = split_dataset["test"]
test_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 350
})

In [11]:
train_dataset = split_dataset["train"]

In [12]:
from datasets import concatenate_datasets

train_dataset = concatenate_datasets([train_dataset, prepared_dataset_2["train"]])
train_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 40650
})

# 4. Training

- Reference: `https://huggingface.co/spaces/openai/whisper/discussions/6`
    - 해당 Reference는 추후에 Dataset 또는 DataLoader를 만들 때 사용하겠다.
 
- 아래는 시간 관계상 3장의 Reference 2개를 적절히 조합하여 복사/붙여넣기를 하였다.

In [13]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 인풋 데이터와 라벨 데이터의 길이가 다르며, 따라서 서로 다른 패딩 방법이 적용되어야 한다. 그러므로 두 데이터를 분리해야 한다.
        # 먼저 오디오 인풋 데이터를 간단히 토치 텐서로 반환하는 작업을 수행한다.
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Tokenize된 레이블 시퀀스를 가져온다.
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 레이블 시퀀스에 대해 최대 길이만큼 패딩 작업을 실시한다.
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 패딩 토큰을 -100으로 치환하여 loss 계산 과정에서 무시되도록 한다.
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라낸다.
        # 해당 토큰은 이후 언제든 추가할 수 있다.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [14]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [15]:
import evaluate

metric_cer = evaluate.load("cer")

In [16]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 0
    cer = 100 * metric_cer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

In [17]:
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="korean", task="transcribe")
model.config.suppress_tokens = []

In [18]:
processor.get_decoder_prompt_ids(language="korean", task="transcribe")

[(1, 50264), (2, 50359), (3, 50363)]

In [20]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-noising_6",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500, # 500
    max_steps=8000, # 4000
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000, # 500
    eval_steps=200, # 100
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",
    greater_is_better=False,
    push_to_hub=True,
)

In [21]:
from huggingface_hub import notebook_login
notebook_login()

In [21]:
from transformers import Seq2SeqTrainer

model.to(device)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [22]:
trainer.train()

C:\Users\Poco\anaconda3\envs\hty\Lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer,Cer
200,0.154800,0.187114,0,6.711883
400,0.186100,0.202672,0,8.597640
600,0.207800,0.213788,0,6.986317
800,0.208300,0.222873,0,9.181793
1000,0.194000,0.223486,0,7.307798
1200,0.195300,0.228450,0,7.441095
1400,0.218800,0.225333,0,8.436900
1600,0.172000,0.228333,0,7.280354
1800,0.176400,0.224279,0,7.280354
2000,0.182100,0.222116,0,7.178422


C:\Users\Poco\anaconda3\envs\hty\Lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\Users\Poco\anaconda3\envs\hty\Lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
C:\Users\Poco\anaconda3\envs\hty\Lib\site-packages\torch\utils\check

TrainOutput(global_step=8000, training_loss=0.09752123728767037, metrics={'train_runtime': 20222.1164, 'train_samples_per_second': 6.33, 'train_steps_per_second': 0.396, 'total_flos': 3.693373670375424e+19, 'train_loss': 0.09752123728767037, 'epoch': 3.15})

In [23]:
kwargs = {
    "dataset": "whisper-kor_noising_6",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ko, split: valid",
    "language": "ko",
    "model_name": "whisper-kor_noising_6",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [24]:
trainer.push_to_hub(**kwargs)

'https://huggingface.co/Taeyeun72/whisper-small-noising_6/tree/main/'

In [22]:
# whisper-small-denoising_all : finetuning with noise and denoise (iter 2, total 8000 steps) (noisy -> denoise) (4번 모델)
# whisper-small-denoising_all_pure: finetuning with only full denoise
# whisper-small-noising_2: finetuning with only noise (iter 2, total 8000 steps)
# whisper-small-noising_3: finetuning with noise and denoise (iter 2, total 8000 steps) (denoise -> noise)
# whisper-small-noising_4: finetuning with only full denoise (iter 2, total 8000 steps)

# whisper-small-noising_5: 4번 모델에 New_Dataset_Denoising(9000개의 데이터)로 파인튜닝
# whisper-small-noising_6: 4번 모델에 Dataset_Denoising_all + New_Dataset_Denoising(6000개의 데이터)
#                          (New_Dataset_Denoising에는 New_Dataset_Denoising의 원본(3000개)의 Combination(3000개)이 포함되어 있다.)
#                          파인튜닝 (iter 3, total 12000 steps)